In [45]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.base import BaseEstimator, TransformerMixin

# Define TextPreprocessor class
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        preprocessed_sentences = []
        for sentence in X:
            # Convert text to lowercase
            sentence = sentence.lower()
                
            # Remove white spaces
            sentence = sentence.strip()
                
            # Remove punctuation
            sentence = sentence.translate(str.maketrans('', '', string.punctuation))
                
            # Remove stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = word_tokenize(sentence)
            sentence = ' '.join([word for word in word_tokens if word not in stop_words])
                
            # Perform stemming
            stemmer = PorterStemmer()
            stemmed_sentence = ' '.join([stemmer.stem(word) for word in word_tokens])
                
            # Perform lemmatization
            lemmatizer = WordNetLemmatizer()
            lemmatized_sentence = ' '.join([lemmatizer.lemmatize(word) for word in word_tokens])
                
            preprocessed_sentences.append((sentence, stemmed_sentence, lemmatized_sentence))
        
        return preprocessed_sentences

# Load the saved model
pickle_file_path = 'models/SA.pkl'
with open(pickle_file_path, 'rb') as f:
    saved_model = pickle.load(f)

# Read the new dataset and drop rows with NaN values
new_data = pd.read_csv("apple_iphone_11_reviews.csv").dropna(subset=["review_text"])

# Instantiate TextPreprocessor
text_preprocessor = TextPreprocessor()

# Preprocess the new dataset
X_new = new_data["review_text"]
preprocessed_new_data = text_preprocessor.transform(X_new)

# Convert preprocessed data into feature vectors using the same CountVectorizer used before
X_new_preprocessed = [sentence[0] for sentence in preprocessed_new_data]  # Assuming you want to use the original preprocessed version
vec = CountVectorizer()
X_new_matrix = vec.fit_transform(X_new_preprocessed)

# Make predictions using the loaded model
y_pred_new = saved_model.predict(X_new_matrix)

# Now you have predictions (y_pred_new) for the new dataset using the saved model


ValueError: X has 4246 features, but LogisticRegression is expecting 47406 features as input.